<a href="https://colab.research.google.com/github/matheshyogeswaran/movie-recommender-with-flask/blob/main/movi_rec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
ratings = pd.read_csv('ratings.dat',
                      sep='::',
                      engine='python',
                      encoding='latin-1',
                      names=['user_id', 'movie_id', 'rating', 'timestamp'],
                      usecols=['user_id', 'movie_id', 'rating'])


users = pd.read_csv('users.dat',
                    sep='::',
                    engine='python',
                    encoding='latin-1',
                    names=['user_id', 'gender', 'age', 'occupation', 'zipcode'],
                    usecols=['user_id', 'gender', 'zipcode', 'age'])

movies = pd.read_csv('movies.dat',
                     sep='::',
                     engine='python',
                     encoding='latin-1',
                     names=['movie_id', 'title', 'genres'],
                     usecols=['movie_id', 'title', 'genres'])


In [ ]:
genre_labels = set()
for s in movies['genres'].str.split('|').values:
    genre_labels = genre_labels.union(set(s))

In [ ]:
def count_word(dataset, ref_col, census):
    keyword_count = dict()
    for s in census:
        keyword_count[s] = 0
    for census_keywords in dataset[ref_col].str.split('|'):
        if type(census_keywords) == float and pd.isnull(census_keywords):
            continue
        for s in [s for s in census_keywords if s in census]:
            if pd.notnull(s):
                keyword_count[s] += 1
    keyword_occurences = sorted(keyword_count.items(), key=lambda x: x[1], reverse=True)
    return keyword_occurences, keyword_count

# Get genre occurrences
keyword_occurences, dum = count_word(movies, 'genres', genre_labels)
print(keyword_occurences[:5])

[('Drama', 1603), ('Comedy', 1200), ('Action', 503), ('Thriller', 492), ('Romance', 471)]


In [ ]:
movies['genres'] = movies['genres'].str.split('|')
movies['genres'] = movies['genres'].fillna("").astype('str')

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=1, stop_words='english')
tfidf_matrix = tf.fit_transform(movies['genres'])

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
print(cosine_sim[:4, :4])

[[1.         0.14193614 0.09010857 0.1056164 ]
 [0.14193614 1.         0.         0.        ]
 [0.09010857 0.         1.         0.1719888 ]
 [0.1056164  0.         0.1719888  1.        ]]


In [ ]:
# Create a mapping from movie titles to indices
titles = movies['title']
indices = pd.Series(movies.index, index=movies['title'])

def genre_recommendations(title):
    idx = indices[title]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:21]
    movie_indices = [i[0] for i in sim_scores]
    return titles.iloc[movie_indices]

In [ ]:
print(genre_recommendations('Toy Story (1995)').head(5))

1050        Aladdin and the King of Thieves (1996)
2072                      American Tail, An (1986)
2073    American Tail: Fievel Goes West, An (1991)
2285                     Rugrats Movie, The (1998)
2286                          Bug's Life, A (1998)
Name: title, dtype: object


In [ ]:
!pip install Flask


In [ ]:
from flask import Flask, jsonify, request
from threading import Thread

app = Flask(__name__)

@app.route('/recommend', methods=['GET'])
def recommend():
    movie_name = request.args.get('movie_name')
    if not movie_name:
        return jsonify({"error": "Movie name is required"}), 400

    try:
        recommendations = genre_recommendations(movie_name)
        return jsonify(recommendations.tolist())
    except KeyError:
        return jsonify({"error": "Movie not found"}), 404

def run():
    app.run(host='0.0.0.0', port=5000)

# Start the app
thread = Thread(target=run)
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:5000
 * Running on http://172.28.0.12:5000
INFO:werkzeug:Press CTRL+C to quit


In [ ]:
!npm install -g localtunnel
!lt --port 5000


added 22 packages, and audited 23 packages in 3s

3 packages are looking for funding
  run `npm fund` for details

1 moderate severity vulnerability

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
your url is: https://metal-poems-sing.loca.lt


INFO:werkzeug:127.0.0.1 - - [12/Oct/2024 18:13:30] "GET /recommend?movie_name=Toy%20Story%20(1995) HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Oct/2024 18:13:41] "GET /recommend?movie_name=Toy%20Story%20(1995) HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [12/Oct/2024 18:15:59] "GET /recommend?movie_name=Toy%20Story%20(19 HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [12/Oct/2024 18:16:06] "GET /recommend?movie_name=Toy%20Story%20(1995) HTTP/1.1" 200 -
